In [5]:
#Get URL's

import requests
import pandas as pd 
import urllib
from bs4 import BeautifulSoup

# settings for the general site
fname = '/Users/emilymartin/Documents/Data_Science/Fluechtlingskrise-Sentiment-Analysis/my_token.txt'
f = open(fname)
my_token = f.read().strip()
f.close()
base_url = 'http://api.zeit.de'
# settings for the specific endpoint
endpoint = 'content'
endpoint_url = f'{base_url}/{endpoint}?api_key={my_token}'
#headers = {'X-Authorization': my_token}

# settings for the specific search query
keyword_search = 'Migranten+Fluechtling AND release_date:[2015-01-01T00:00:00Z TO 2015-12-31T23:59:59.999Z]' # search for all occurrences of 'Migrant' from 2015
limit = 100 # limit can't be over 1000 I think, but 100 should do it


# pass the settings and parameters to the get request
search_params = {'q':keyword_search, 'limit':limit} # see Zeit site for additional options http://developer.zeit.de/docs/content/
response = requests.get(endpoint_url, params=search_params) # send the request!

# look at the keys for the json dictionary produced by the pull request
print(response.json().keys())

# the data itself is in 'matches'. Select the matches element from the dictionary and convert it to a dataframe:
df = pd.DataFrame(response.json()['matches'])

#print(f'Your options are: {df.columns}')
#print(df[['title','href','release_date']])


df

dict_keys(['matches', 'found', 'limit', 'offset'])


,subtitle,uuid,title,href,release_date,uri,snippet,supertitle,teaser_text,teaser_title
0,Ein deutscher Führerschein ist für viele Flüch...,7ykdw6fWIZkfEoCA5HrHKJ,Mahmood im Schilderwald,http://www.zeit.de/2015/51/fuehrerschein-fluec...,2015-12-31T02:51:37Z,http://api.zeit.de/content/7ykdw6fWIZkfEoCA5HrHKJ,". ""Wer Auto fahren kann, hat beruflich viel me...",Führerschein für Flüchtlinge,Ein deutscher Führerschein ist für viele Flüch...,Mahmood im Schilderwald
1,Atticus Lish erzählt souverän von der unwahrsc...,4wLrvdk6M6AKGig80RgjfP,Zwei zähe Einzelgänger,http://www.zeit.de/2015/51/vorbereitung-auf-da...,2015-12-31T01:56:01Z,http://api.zeit.de/content/4wLrvdk6M6AKGig80RgjfP,Kriegserlebnisse sich an die Oberfläche arbei...,Atticus Lish,Atticus Lish erzählt souverän von der unwahrsc...,Zwei zähe Einzelgänger
2,"Von Container-Millionären, Pegida-Anführerinne...",1t6ENxcNNbG830MqMoPiTB,Fortsetzung folgt – jetzt,http://www.zeit.de/2016/01/geschichten-2015-fo...,2015-12-30T09:00:08Z,http://api.zeit.de/content/1t6ENxcNNbG830MqMoPiTB,Bachmann hatten sich auseinandergelebt. Sie s...,Geschichten 2015,"Von Container-Millionären, Pegida-Anführerinne...",Fortsetzung folgt – jetzt
3,Nach der Grenzschließung in Ungarn führt die F...,5a6YZ9hEdbbfjhc2YQXnDP,Anhaltend hohe Flüchtlingszahlen auf Balkanroute,http://www.zeit.de/gesellschaft/2015-12/slowen...,2015-12-29T22:14:02Z,http://api.zeit.de/content/5a6YZ9hEdbbfjhc2YQXnDP,Auch zum Jahresende kommen weiter täglich Taus...,Slowenien,Nach der Grenzschließung in Ungarn führt die F...,Anhaltend hohe Flüchtlingszahlen auf Balkanroute
4,Integrationspflicht für Flüchtlinge? In der De...,1EN443vyRBFWFwyCqHQbTJ,Laut Özoğuz schürt Union Vorurteile gegen Flüc...,http://www.zeit.de/politik/deutschland/2015-12...,2015-12-29T08:24:55Z,http://api.zeit.de/content/1EN443vyRBFWFwyCqHQbTJ,Opposition und Koalitionspartner kritisieren d...,Integration,Integrationspflicht für Flüchtlinge? In der De...,Laut Özoğuz schürt Union Vorurteile gegen Flüc...
...,...,...,...,...,...,...,...,...,...,...
95,Es geschah Dramatisches: Terroralarm bei der N...,49puVFglhhIqqByaDLTNta,Ein Abend ohne Vorkommnisse,http://www.zeit.de/sport/2015-11/fussball-nati...,2015-11-18T05:48:07Z,http://api.zeit.de/content/49puVFglhhIqqByaDLTNta,Generalverdacht gegen <em>Migranten</em> an. ...,Hannover,Es geschah Dramatisches: Terroralarm bei der N...,Ein Abend ohne Vorkommnisse
96,Gegen Dschihadisten helfen meist nur noch Poli...,6VPq3iAmzftQ0tSRSlGdkJ,"Integration und Prävention, jetzt erst recht",http://www.zeit.de/gesellschaft/zeitgeschehen/...,2015-11-17T14:14:38Z,http://api.zeit.de/content/6VPq3iAmzftQ0tSRSlGdkJ,Heute ist Integrationsgipfel. Aber wen interes...,Integrationsgipfel,Gegen Dschihadisten helfen meist nur noch Poli...,"Integration und Prävention, jetzt erst recht"
97,Die Vereinten Nationen erheben massive Vorwürf...,1zdJgG01Sko7QjhaGOQnvd,Flüchtlinge werden laut UN in Libyen gefoltert,http://www.zeit.de/politik/ausland/2015-11/men...,2015-11-17T00:58:08Z,http://api.zeit.de/content/1zdJgG01Sko7QjhaGOQnvd,"<em>Migranten</em>, die in Haftlagern in Libye...",Menschenrechte,Die Vereinten Nationen erheben massive Vorwürf...,Flüchtlinge werden laut UN in Libyen gefoltert
98,"Sieben junge Wiener erzählen, wo sie sich selb...",3qBEDSlutcUsBNeINiQB2h,Wie geht’s weiter?,http://www.zeit.de/2015/46/wiener-zukunft-prot...,2015-11-16T09:00:03Z,http://api.zeit.de/content/3qBEDSlutcUsBNeINiQB2h,"könnte alles sein. Ich weiß, dass jetzt das B...",Österreich 2025,"Sieben junge Wiener erzählen, wo sie sich selb...",Wie geht’s weiter?


In [4]:
print(response.json())

{'matches': [{'subtitle': 'Ein deutscher Führerschein ist für viele Flüchtlinge die Eintrittskarte zum Arbeitsmarkt. Doch die Hürden sind hoch.', 'uuid': '7ykdw6fWIZkfEoCA5HrHKJ', 'title': 'Mahmood im Schilderwald', 'href': 'http://www.zeit.de/2015/51/fuehrerschein-fluechtinge-auto-chancen', 'release_date': '2015-12-31T02:51:37Z', 'uri': 'http://api.zeit.de/content/7ykdw6fWIZkfEoCA5HrHKJ', 'snippet': '. "Wer Auto fahren kann, hat beruflich viel mehr Chancen", sagt er. Tatsächlich scheinen Jobs wie Taxifahrer, Paketzusteller oder Pizzabote für <em>Migranten</em> ohne Ausbildung naheliegend. Und wer auf dem Land', 'supertitle': 'Führerschein für Flüchtlinge', 'teaser_text': 'Ein deutscher Führerschein ist für viele Flüchtlinge die Eintrittskarte zum Arbeitsmarkt. Doch die Hürden sind hoch.', 'teaser_title': 'Mahmood im Schilderwald'}, {'subtitle': 'Atticus Lish erzählt souverän von der unwahrscheinlichen Liebesgeschichte zwischen einem Irak-Veteranen und einer illegalen Immigrantin aus C

In [ ]:
#Get all the URL's into a list
url_lst = df.href.tolist()
#lst_short = url_lst[:5]
len(url_lst)

In [ ]:

art_dict = {} #Initiate empty dictionary 
x = ' ' #For joining the text we are collecting
for u in url_lst:                 #Loop through each url and use Beautiful Soup to get the parts we need
    page = urllib.request.urlopen(u)                     
    soup = BeautifulSoup(page)
    t = soup.find("div", {"class":"article-body"})           #Luckily all the tags are the same for each article
    paras = t.find_all("p")           #Within this div all p's hold the article text
    a = [p.text for p in paras]           #Nifty list comprehension to turn the bs4 object into text
    text = x.join(a)      #Join text
    art_dict[u] = text                #Add to dictionary with keys as URL's and values as text

art_dict
#tag = <div class="article-body article-body--article">

In [ ]:
#Map the text to a new column in our dataframe using the href keys, which are also a column in df. How cool!
df['text']= df['href'].map(art_dict)    
df.head()

In [ ]:
#Clean data

In [ ]:
#Analyse